In [1]:
import requests
import pandas as pd
import json
from tqdm import tqdm 
from time import sleep

In [22]:
with open ('./ufs.json', 'r', encoding='utf8') as f:
    ufs = json.load(f)

In [34]:
orgs_final = []
for uf in tqdm(ufs):
    u = uf.get('sigla')
    url = f"https://sgip3.tse.jus.br/sgip3-consulta/api/v1/orgaoPartidario/consulta?dataFimVigencia=31%2F10%2F2023&dataInicioVigencia=01%2F06%2F2017&isComposicoesHistoricas=true&nrZona=0&sgUe={u}&sqPartido=0&tpAbrangencia=82"
    while True:
        try:
            resp = requests.get(url)
            break
        except requests.exceptions.ConnectionError:
            sleep(5)
            print("sleeping")
    orgs = resp.json()
    orgs_final += orgs

  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 26/26 [00:43<00:00,  1.68s/it]


In [35]:
pd.DataFrame(orgs_final).to_excel('orgaos.xlsx')

In [2]:
df_orgaos = pd.read_excel('orgaos.xlsx')
orgs_final = df_orgaos.to_dict('records')

In [ ]:
membros_final = []
for org in tqdm(orgs_final):
    org_id = org.get('sqOrgaoPartidario') 
    url_org = f"https://sgip3.tse.jus.br/sgip3-consulta/api/v1/orgaoPartidario/comAnotacoesEMembros?idOrgaoPartidario={org_id}"
    while True:
        try:
            resp = requests.get(url_org)
            if resp.status_code == 200:
                membros = resp.json().get('membros')
                for membro in membros:
                    membro['orgaoId'] = org_id
                membros_final += membros
            break
        except requests.exceptions.ConnectionError:
            print('sleeping')
            sleep(5)
    

In [19]:
df_membros = pd.DataFrame(membros_final)
df_membros.to_excel('membros.xlsx')

In [8]:
df_membros.merge(df_orgaos, left_on='orgaoId', right_on='sqOrgaoPartidario').to_excel('membros_orgaos.xlsx')